# Final Python Project

This is a software to manage a vegan product shop.
Provided functionalities are:
1. add a new product in warehouse
1. list all products in warehouse
1. record all sales
1. display gross and net earnings from sales

PS the program language is italian




_Technical details_

To execute the program, follow these steps:
1. Inside project directory, create a python virtual environment with venv module:
> python -m venv myenv
1. Activate the virtual environment (it depends on which OS you're working on, [see doc](https://docs.python.org/3/library/venv.html)).
1. Download and install all required packages and modules with
> pip install -r requirements.txt
1. Create a kernel inside virtual env with
>python -m ipykernel install --user --name=myenv_kernel
1. Open "main.ipynb" with Jupyter notebook, with new kernel.
1. Run the two code blocks below and follow terminal instructions.

During execution two csv files will be created in the main project directory (warehouse.csv and sales.csv). Don't delete them in order to keep operations' persistence.

In [1]:
import re

def show_help():
    """
    This function shows an help message about how to use this software.
    """
    
    print("\n\nI comandi disponibili sono i seguenti:")
    print("aggiungi: aggiungi un prodotto al magazzino")
    print("elenca: elenca i prodotto in magazzino")
    print("vendita: registra una vendita effettuata")
    print("profitti: mostra i profitti totali")
    print("aiuto: mostra i possibili comandi")
    print("chiudi: esci dal programma\n")

    
def is_positive_integer(value_str):
    """
    Check if a string is a positive integer, different than zero.
    Input:
    - value_str (str): any string. 
    Ouput:
    - bool value determining if value_str is a positive integer or not.
    """
    pattern = re.compile(r'^\d+$')
    if pattern.match(value_str):
        return int(value_str) != 0
    else:
        return False

    
def is_positive_float(value_str):
    """
    Check if a string is a positive float, different than zero.
    Input:
    - value_str (str): any string. 
    Ouput:
    - bool value determining if value_str is a positive float or not.
    """
    pattern = re.compile(r'^\d+(\.\d+)?$')
    if pattern.match(value_str):
        return float(value_str) != 0
    else:
        return False


def input_positive_int(variable_name):
    """
    Read a string from keyboard and parse it to a positive integer. 
    If user inputs an invalid value, the function asks again to
    insert a new value.
    Input:
    - variable_name (str): variable's name, used in input prompt.
    Ouput:
    - value (int): positive int value, parsed from string inserted 
    from keyboard.
    """
    while True:
        value_str = input(f"{variable_name}: ")
        if(is_positive_integer(value_str)):
            value = int(value_str)
            return value
        print("Input non valido. Inserisci un numero intero positivo.")
    

def input_positive_float(variable_name):
    """
    Read a string from keyboard and parse it to a positive float. 
    If user inputs an invalid value, the function asks again to
    insert a new value.
     Input:
    - variable_name (str): variable's name, used in input prompt.
    Ouput:
    - value (float): positive float value, parsed from string inserted 
    from keyboard.
    """
    while True:
        value_str = input(f"{variable_name}: ")
        if(is_positive_float(value_str)):
            value = float(value_str)
            return value
        print("Input non valido. Inserisci un numero decimale positivo.")
    

In [2]:
import warehouse_utils
import sales_utils
from Product import *

warehouse_utils.file_init()
sales_utils.file_init()

print("Benvenuto nel gestionale del negozio.")
show_help()

cmd = None

while cmd!="chiudi":
    try:

      cmd = input("\n\nInserisci un comando: ")

      if cmd=="vendita":
        # record a sale
        sale_details=[]
        add_item = None
        while(add_item != "no"):

            product_name = input("Nome del prodotto: ")
            sale_quantity = input_positive_int("Quantità")

            #check if product is in warehouse
            p = warehouse_utils.get_item(product_name)
            if p==None:
                print(f"Prodotto {product_name} non presente a magazzino.")
            else:
                if(p.get_quantity() < sale_quantity):
                    print(f"Disponibilità massima: {p.get_quantity()}")
                    sale_quantity = p.get_quantity()

                if sale_quantity == 0:
                    print("Prodotto non disponibile")
                else:
                    warehouse_utils.update_product_quantity(p.get_name(),-sale_quantity)

                    sale = [round(p.get_sale_price()*sale_quantity,2), round(p.get_purchase_price()*sale_quantity,2)]
                    sales_utils.add(sale)
                    sale_details.append({"product" : p.get_name(), 
                                         "quantity":sale_quantity,
                                         "price":p.get_sale_price()})
            add_item = input("Aggiungere un altro prodotto? (si/no) ")

        #at least one item sold.
        if len(sale_details) > 0:
            print("VENDITA REGISTRATA")
            total_transaction = 0.0
            for detail in sale_details:
                print(f"{detail['quantity']}x {detail['product']}: €{detail['price']:.2f}")
                total_transaction += detail["price"]*detail["quantity"]
            print(f"Totale: €{total_transaction:.2f}")


      elif cmd=="profitti":
        # display net and gross earnings
        earnings = sales_utils.calculate_earnings()
        print(f"Profitto: lordo=€{earnings[0]:.2f} netto=€{earnings[1]:.2f}")


      elif cmd=="aggiungi":
        # add an item to warehouse

        name = input("Nome del prodotto: ")
        quantity = input_positive_int("Quantità")
        
        #check if product is in warehouse
        p = warehouse_utils.get_item(name)

        if p == None:
            #new product
            purchase_price = input_positive_float("Prezzo di acquisto")
            sale_price = input_positive_float("Prezzo di vendita")

            p = Product(name,sale_price,purchase_price,quantity)
            warehouse_utils.add(p)

        else:
            print("Prodotto già registrato nel magazzino.")
            warehouse_utils.update_product_quantity(name,quantity)  
        print(f"AGGIUNTO: {quantity}x {name}")

      elif cmd=="elenca":
        # list all products in warehouse
        products_list = warehouse_utils.get_all()
        print("{:<15} | {} | {}".format("Prodotto","Quantità","Prezzo"))
        for p in products_list:
            print(p)

      elif cmd=="aiuto":
        # show supported commands
        show_help()

      elif cmd=="chiudi":
        # say goodbye and stop the program
        print("Arrivederci")

      else:
        # not valid command
        # show help message
        print("Comando non valido")
        show_help()
    except Exception as e:
        print("ERRORE: ",e)

Benvenuto nel gestionale del negozio.


I comandi disponibili sono i seguenti:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotto in magazzino
vendita: registra una vendita effettuata
profitti: mostra i profitti totali
aiuto: mostra i possibili comandi
chiudi: esci dal programma



Inserisci un comando: elenca
Prodotto        | Quantità | Prezzo
tofu            | 7        | 4.19
seitan          | 10       | 5.49
latte di soia   | 13       | 1.40
yogurt          | 7        | 1.50
verza           | 10       | 6.00
orzo            | 4        | 9.00
pomodoro        | 6        | 0.80
segale          | 2        | 1.20


Inserisci un comando: profitti
Profitto: lordo=€127.08 netto=€59.58


Inserisci un comando: aiuto


I comandi disponibili sono i seguenti:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotto in magazzino
vendita: registra una vendita effettuata
profitti: mostra i profitti totali
aiuto: mostra i possibili comandi
chiudi: esci dal programma
